In [1]:
pip install transformers opencv-python pandas pillow


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB 435.7 kB/s eta 0:01:31
   ---------------------------------------- 0.1/39.5 MB 653.6 kB/s eta 0:01:01
   ---------------------------------------- 0.3/39.5 MB 1.9 MB/s eta 0:00:21
   - -------------------------------------- 1.2/39.5 MB 6.1 MB/s eta 0:00:07
   --- ------------------------------------ 3.4/39.5 MB 14.4 MB/s eta 0:00:03
   ----- ---------------------------------- 5.0/39.5 MB 17.6 MB/s eta 0:00:02
   ------- -------------------------------- 7.3/39.5 MB 22.4 MB/s eta 0:00:02
   ---------- ----------------------------- 10.7/39.5 MB 40.9 MB/s eta 0:00:01
   ------------ --------------------------- 12.7/39.5 MB 43.7 MB/s eta 0:00:01
   --------------- ------------------------ 15.1/39.5 MB 46.9 MB/s eta 0:00:01
   ----------------- ---------------------- 17.1/39.5 MB 43.5 MB/s eta 0:00:0

In [6]:
!pip install timm


     ---------------------------------------- 0.0/52.0 kB ? eta -:--:--
     ------- -------------------------------- 10.2/52.0 kB ? eta -:--:--
     ------- -------------------------------- 10.2/52.0 kB ? eta -:--:--
     ----------------------------- -------- 41.0/52.0 kB 245.8 kB/s eta 0:00:01
     -------------------------------------- 52.0/52.0 kB 242.9 kB/s eta 0:00:00
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   - -------------------------------------- 0.1/2.4 MB 2.3 MB/s eta 0:00:01
   ----------- ---------------------------- 0.7/2.4 MB 8.9 MB/s eta 0:00:01
   ------------------------------- -------- 1.8/2.4 MB 14.6 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 15.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 36.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/212.5 MB ? eta -:--:--
   ---------------------------

  You can safely remove it manually.


In [1]:
import cv2
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
from datetime import datetime
from transformers import pipeline
import os

detector = pipeline("object-detection", model="facebook/detr-resnet-50")

output_dir = "output"
os.makedirs(output_dir, exist_ok=True)


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

C:\Users\tanna\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tanna\.cache\huggingface\hub\models--timm--resnet50.a1_in1k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['

preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

C:\Users\tanna\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\tanna\.cache\huggingface\hub\models--facebook--detr-resnet-50. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [3]:

def frame_to_pil(frame):
    """Convert OpenCV BGR frame to PIL RGB image"""
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return Image.fromarray(rgb)

def draw_boxes(frame, detections):
    """Draw bounding boxes on the frame using PIL"""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    pil_image = Image.fromarray(frame_rgb)
    draw = ImageDraw.Draw(pil_image)
    font = ImageFont.load_default()

    for det in detections:
        box = det["box"]
        label = det["label"]
        score = det["score"]
        
        xmin, ymin = int(box["xmin"]), int(box["ymin"])
        xmax, ymax = int(box["xmax"]), int(box["ymax"])
        
        draw.rectangle([(xmin, ymin), (xmax, ymax)], outline="red", width=3)
        text = f"{label} {score:.2f}"
        
        text_bbox = draw.textbbox((xmin, ymin), text, font=font)
        draw.rectangle([text_bbox[0:2], text_bbox[2:4]], fill="red")
        draw.text((xmin, ymin), text, fill="white", font=font)

    return cv2.cvtColor(np.array(pil_image), cv2.COLOR_RGB2BGR)


In [5]:

def process_video(video_path, target_label="carrot", output_name="carrot_output.mp4"):
    """
    Processes the video and detects a target object (e.g., 'carrot') frame by frame.
    Saves a video with bounding boxes and a CSV log of detections.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError("Cannot open video file")

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Output setup
    output_video_path = os.path.join(output_dir, output_name)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    carrot_log = []
    frame_num = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_num += 1
        pil_img = frame_to_pil(frame)
        results = detector(pil_img)

        carrot_detections = [r for r in results if target_label.lower() in r['label'].lower()]
        if carrot_detections:
            for det in carrot_detections:
                carrot_log.append({
                    "frame": frame_num,
                    "time": datetime.now().isoformat(),
                    "label": det['label'],
                    "score": det['score'],
                    "box": det['box']
                })

        # Draw boxes and write frame
        annotated_frame = draw_boxes(frame, carrot_detections)
        out.write(annotated_frame)

        # Print progress every 30 frames
        if frame_num % 30 == 0:
            print(f"Processed frame {frame_num}/{total}")

    cap.release()
    out.release()

    # Save log
    df = pd.DataFrame(carrot_log)
    log_path = os.path.join(output_dir, "carrot_detections.csv")
    df.to_csv(log_path, index=False)

    print(f"\n✅ Processing complete. Output saved to:")
    print(f"- Video: {output_video_path}")
    print(f"- Log:   {log_path}")


In [7]:

input_video_path = "carrot.mp4"  # <-- Update if needed

process_video(input_video_path, target_label="carrot")


Processed frame 30/14550
Processed frame 60/14550
Processed frame 90/14550
Processed frame 120/14550
Processed frame 150/14550
Processed frame 180/14550
Processed frame 210/14550
Processed frame 240/14550
Processed frame 270/14550
Processed frame 300/14550
Processed frame 330/14550
Processed frame 360/14550
Processed frame 390/14550
Processed frame 420/14550
Processed frame 450/14550
Processed frame 480/14550
Processed frame 510/14550
Processed frame 540/14550
Processed frame 570/14550
Processed frame 600/14550
Processed frame 630/14550
Processed frame 660/14550
Processed frame 690/14550
Processed frame 720/14550
Processed frame 750/14550
Processed frame 780/14550
Processed frame 810/14550
Processed frame 840/14550
Processed frame 870/14550
Processed frame 900/14550
Processed frame 930/14550
Processed frame 960/14550
Processed frame 990/14550
Processed frame 1020/14550
Processed frame 1050/14550
Processed frame 1080/14550
Processed frame 1110/14550
Processed frame 1140/14550
Processed 